# making an aggregate master dataframe for baseline model
this notebook is going to make a single standardized long format dataframe for each of my cleaned clinical variable dataframes.

will save this into a file with a cleaned_merged_df prefix.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

time: 0 ns (started: 2023-07-26 19:22:14 +08:00)


In [3]:
#patients of interest from rotation_cohort_generation
from parameters import final_pt_df_v, date, repository_path

#patients of interest from rotation_cohort_generation
final_pt_df2 = final_pt_df_v #pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_final_pt_df2.csv'%(most_updated_patient_df), index_col=0)
del(final_pt_df_v)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['stay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 16 ms (started: 2023-07-26 19:22:40 +08:00)


In [5]:
final_pt_df2['stay_id'].nunique() #26471

26471

time: 0 ns (started: 2023-07-26 19:22:47 +08:00)


In [6]:
from parameters import lower_window, upper_window, folder, date, time_col, time_var, patient_df

time: 0 ns (started: 2023-07-26 19:22:55 +08:00)


In [7]:
#import all cleaned or 'prepped' clinical variable csv
allFiles = glob.glob(str(repository_path)+ '/data/cleaned/%s/'%(folder) + "{}_*.csv".format(date))
allFiles

['c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/cleaned/24_hr_window\\07242023_all_vaso_prepped.csv',
 'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/cleaned/24_hr_window\\07242023_bg_all_prepped.csv',
 'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/cleaned/24_hr_window\\07242023_bg_ART_prepped.csv',
 'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/cleaned/24_hr_window\\07242023_cancer_elix_prepped.csv',
 'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/cleaned/24_hr_window\\07242023_dobutamine_prepped.csv',
 'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/cleaned/24_hr_window\\07242023_dopamine_prepped.csv',
 'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/cleaned/24_hr_window\\07242023_epinephrine_prepped.csv',
 'c:\\Users\\csfhe\\Documents\\RA\\mi

time: 15 ms (started: 2023-07-26 19:23:01 +08:00)


In [8]:
save_path= str(repository_path)+'/data/cleaned_merged/'
def save_df(df, df_name='default', save_path=save_path, add_subfolder=False):
    #uses the date and supplied df name and saves to the savepath specified above.
    if df_name == 'default':
        df_name= "%s"%(df)
    
    address=save_path+'%s/'%(folder)
    if not os.path.exists(address):
        print(address)
        os.makedirs(address)
    pd.DataFrame(df).to_csv(Path(address+'%s_%s_cleaned_merged.csv' %(date, df_name)))

time: 0 ns (started: 2023-07-26 19:23:05 +08:00)


In [9]:
os.getcwd()

'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks'

time: 0 ns (started: 2023-07-26 19:23:15 +08:00)


In [10]:
#making a dictionary of all my dataframes for easier cycling through

df_list=[]
for element in allFiles:
    df_list.append(element.split('{}_'.format(date))[1].split('_prepped.csv')[0]) #making an list of all my dataframes in order they appear in file

dfs = {}
i=0
for name in df_list:
    dfs[name] = pd.read_csv(allFiles[i],  index_col=0)
    i+=1

C:\Users\csfhe\AppData\Local\Temp\ipykernel_16288\873374919.py:10: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[name] = pd.read_csv(allFiles[i],  index_col=0)
C:\Users\csfhe\AppData\Local\Temp\ipykernel_16288\873374919.py:10: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[name] = pd.read_csv(allFiles[i],  index_col=0)


time: 8.52 s (started: 2023-07-26 19:23:16 +08:00)


In [11]:
len(dfs) # 24 -> 21

21

time: 0 ns (started: 2023-07-26 19:23:41 +08:00)


In [12]:
#all of the column names 
for element in df_list:
    print(element,':',list(dfs[element]))

all_vaso : ['stay_id', 't_0', 'value', 'label', 'uom', 'delta']
bg_all : ['subject_id', 'hadm_id', 'stay_id', 'charttime', 'delta', 't_0', 'label', 'valuenum', 'uom']
bg_ART : ['delta', 'hadm_id', 'label', 'stay_id', 'subject_id', 't_0', 'uom', 'value']
cancer_elix : ['stay_id', 't_0', 'value', 'label', 'uom', 'delta']
dobutamine : ['stay_id', 't_0', 'value', 'label', 'uom', 'delta']
dopamine : ['stay_id', 't_0', 'value', 'label', 'uom', 'delta']
epinephrine : ['stay_id', 't_0', 'value', 'label', 'uom', 'delta']
gcs : ['subject_id', 'hadm_id', 'stay_id', 'delta', 'label', 'value', 't_0', 'uom']
height : ['stay_id', 'value', 'uom', 'label', 'delta', 't_0']
labs : ['subject_id', 'hadm_id', 'stay_id', 'charttime', 'label', 'value', 'uom', 't_0', 'delta']
norepinephrine : ['stay_id', 't_0', 'value', 'label', 'uom', 'delta']
o2_flow : ['stay_id', 't_0', 'value', 'label', 'uom', 'delta']
pfRatio : ['subject_id', 'hadm_id', 'stay_id', 'value', 'delta', 'uom', 'label']
phenylephrine : ['stay_i

## standardizing columns

#### adding icustay_id, dropping hadm_id

In [13]:
##dropping hadm_id from all:
list1=[]
for element in df_list:
    if 'hadm_id' in (list(dfs[element])):
        list1.append(element)

for element in list1:
    dfs[element]= dfs[element].drop('hadm_id', axis=1)

time: 313 ms (started: 2023-07-26 19:24:18 +08:00)


In [14]:
##dropping subject_id from all:
list1=[]
for element in df_list:
    if 'subject_id' in (list(dfs[element])):
        list1.append(element)

for element in list1:
    dfs[element]= dfs[element].drop('subject_id', axis=1)

time: 297 ms (started: 2023-07-26 19:24:20 +08:00)


In [15]:
#all of the column names 
for element in df_list:
    print(element,':',sorted(list(dfs[element])))

all_vaso : ['delta', 'label', 'stay_id', 't_0', 'uom', 'value']
bg_all : ['charttime', 'delta', 'label', 'stay_id', 't_0', 'uom', 'valuenum']
bg_ART : ['delta', 'label', 'stay_id', 't_0', 'uom', 'value']
cancer_elix : ['delta', 'label', 'stay_id', 't_0', 'uom', 'value']
dobutamine : ['delta', 'label', 'stay_id', 't_0', 'uom', 'value']
dopamine : ['delta', 'label', 'stay_id', 't_0', 'uom', 'value']
epinephrine : ['delta', 'label', 'stay_id', 't_0', 'uom', 'value']
gcs : ['delta', 'label', 'stay_id', 't_0', 'uom', 'value']
height : ['delta', 'label', 'stay_id', 't_0', 'uom', 'value']
labs : ['charttime', 'delta', 'label', 'stay_id', 't_0', 'uom', 'value']
norepinephrine : ['delta', 'label', 'stay_id', 't_0', 'uom', 'value']
o2_flow : ['delta', 'label', 'stay_id', 't_0', 'uom', 'value']
pfRatio : ['delta', 'label', 'stay_id', 'uom', 'value']
phenylephrine : ['delta', 'label', 'stay_id', 't_0', 'uom', 'value']
pt_info : ['delta', 'label', 'stay_id', 't_0', 'uom', 'value']
rrt : ['delta', '

In [16]:
#dropping charttime, endtime and first_charttime
list1=[]
list2=[]
for element in df_list:
    if 'charttime' in (list(dfs[element])):
        list1.append(element)
    if 'endtime' in (list(dfs[element])):
        list2.append(element)

for element in list1:
    dfs[element]= dfs[element].drop('charttime', axis=1)

for element in list2:
    dfs[element]= dfs[element].drop('endtime', axis=1)
    
#dfs['rrt']= dfs['rrt'].drop('first_charttime', axis=1)

time: 250 ms (started: 2023-07-26 19:25:37 +08:00)


In [17]:
#converting valuenum and value to same label
list1=[]

for element in df_list:
    if 'valuenum' in (list(dfs[element])):
        list1.append(element)

for element in list1:
    dfs[element]= dfs[element].rename(index=str, columns={'valuenum':'value'})


time: 94 ms (started: 2023-07-26 19:25:39 +08:00)


In [18]:
del(list1,list2)

time: 0 ns (started: 2023-07-26 19:25:42 +08:00)


In [19]:
def label_lower(df_name):
    dfs[df_name]['label']= dfs[df_name]['label'].apply(lambda x: x.lower())

time: 0 ns (started: 2023-07-26 19:25:44 +08:00)


In [20]:
#turning all labels to lowercase
for element in df_list:
    label_lower(element)


time: 1.2 s (started: 2023-07-26 19:25:46 +08:00)


In [22]:
#adding a df source table label to each df. 
for element in df_list:
    dfs[element]['source']=element

#adding a patient id to each

for element in df_list:
    dfs[element]= pd.merge(dfs[element], final_pt_df2[['stay_id','subject_id']], how='left')
    

time: 1.16 s (started: 2023-07-26 19:25:58 +08:00)


In [23]:
#all of the column names 
for element in df_list:
    print(element,':',sorted(list(dfs[element])))

all_vaso : ['delta', 'label', 'source', 'stay_id', 'subject_id', 't_0', 'uom', 'value']
bg_all : ['delta', 'label', 'source', 'stay_id', 'subject_id', 't_0', 'uom', 'value']
bg_ART : ['delta', 'label', 'source', 'stay_id', 'subject_id', 't_0', 'uom', 'value']
cancer_elix : ['delta', 'label', 'source', 'stay_id', 'subject_id', 't_0', 'uom', 'value']
dobutamine : ['delta', 'label', 'source', 'stay_id', 'subject_id', 't_0', 'uom', 'value']
dopamine : ['delta', 'label', 'source', 'stay_id', 'subject_id', 't_0', 'uom', 'value']
epinephrine : ['delta', 'label', 'source', 'stay_id', 'subject_id', 't_0', 'uom', 'value']
gcs : ['delta', 'label', 'source', 'stay_id', 'subject_id', 't_0', 'uom', 'value']
height : ['delta', 'label', 'source', 'stay_id', 'subject_id', 't_0', 'uom', 'value']
labs : ['delta', 'label', 'source', 'stay_id', 'subject_id', 't_0', 'uom', 'value']
norepinephrine : ['delta', 'label', 'source', 'stay_id', 'subject_id', 't_0', 'uom', 'value']
o2_flow : ['delta', 'label', 'sou

In [25]:
for element in df_list:
    print(element,'column that dont belong:', [x for x in list(dfs[element]) if x not in ['delta', 'stay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']])
    print(element,'column that are missing:',[x for x in ['delta', 'stay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value'] if x not in list(dfs[element])])

all_vaso column that dont belong: []
all_vaso column that are missing: []
bg_all column that dont belong: []
bg_all column that are missing: []
bg_ART column that dont belong: []
bg_ART column that are missing: []
cancer_elix column that dont belong: []
cancer_elix column that are missing: []
dobutamine column that dont belong: []
dobutamine column that are missing: []
dopamine column that dont belong: []
dopamine column that are missing: []
epinephrine column that dont belong: []
epinephrine column that are missing: []
gcs column that dont belong: []
gcs column that are missing: []
height column that dont belong: []
height column that are missing: []
labs column that dont belong: []
labs column that are missing: []
norepinephrine column that dont belong: []
norepinephrine column that are missing: []
o2_flow column that dont belong: []
o2_flow column that are missing: []
pfRatio column that dont belong: []
pfRatio column that are missing: ['t_0']
phenylephrine column that dont belong: 

In [26]:
pd.to_timedelta(dfs['norepinephrine']['delta']).describe()

count                        26471
mean     0 days 01:35:13.708586755
std      0 days 04:07:46.276158603
min                0 days 00:00:00
25%                0 days 00:00:00
50%                0 days 00:00:00
75%                0 days 00:00:00
max                1 days 00:00:00
Name: delta, dtype: object

time: 78 ms (started: 2023-07-26 19:26:46 +08:00)


# converting data formats

### looking at measured values (quick qc)

In [28]:
len(df_list)

21

time: 16 ms (started: 2023-07-26 19:26:54 +08:00)


In [29]:
def value_viewer(df_name):
    return(dfs[df_name]['label'].unique())

time: 0 ns (started: 2023-07-26 19:26:57 +08:00)


In [30]:
value_viewer('bg_ART')

array(['po2', 'pco2'], dtype=object)

time: 0 ns (started: 2023-07-26 19:27:03 +08:00)


In [31]:
value_viewer('bg_all')

array(['lactate', 'ph', 'fio2', 'temperature', 'calcium', 'hemoglobin',
       'glucose', 'chloride', 'potassium', 'sodium', 'bicarbonate'],
      dtype=object)

time: 15 ms (started: 2023-07-26 19:27:11 +08:00)


In [27]:
#value_viewer('uti')

time: 963 µs


In [32]:
value_viewer('labs')


array(['hemoglobin', 'platelet', 'wbc', 'bicarbonate', 'bilirubin',
       'chloride', 'creatinine', 'glucose', 'potassium', 'sodium', 'bun',
       'lactate', 'inr', 'ptt', 'bands'], dtype=object)

time: 47 ms (started: 2023-07-26 19:27:19 +08:00)


In [33]:
value_viewer('vitals')

array(['resprate', 'temperature', 'heartrate', 'spo2', 'sysbp', 'diasbp',
       'meanartpress', 'glucose'], dtype=object)

time: 219 ms (started: 2023-07-26 19:27:22 +08:00)


In [34]:
value_viewer('pt_info')

array(['yearsold', 'gender', 'race'], dtype=object)

time: 16 ms (started: 2023-07-26 19:27:25 +08:00)


In [35]:
list(dfs)

['all_vaso',
 'bg_all',
 'bg_ART',
 'cancer_elix',
 'dobutamine',
 'dopamine',
 'epinephrine',
 'gcs',
 'height',
 'labs',
 'norepinephrine',
 'o2_flow',
 'pfRatio',
 'phenylephrine',
 'pt_info',
 'rrt',
 'sum_elix',
 'vasopressin',
 'ventcategory',
 'vitals',
 'weight']

time: 0 ns (started: 2023-07-26 19:27:28 +08:00)


# combining data

In [36]:
set(value_viewer('labs')) & set(value_viewer('bg_all'))

{'bicarbonate',
 'chloride',
 'glucose',
 'hemoglobin',
 'lactate',
 'potassium',
 'sodium'}

time: 78 ms (started: 2023-07-26 19:27:33 +08:00)


In [37]:
set(value_viewer('labs')) & set(value_viewer('vitals'))

{'glucose'}

time: 265 ms (started: 2023-07-26 19:27:36 +08:00)


In [38]:
set(value_viewer('bg_all')) & set(value_viewer('vitals'))

{'glucose', 'temperature'}

time: 235 ms (started: 2023-07-26 19:27:38 +08:00)


# combining all df

In [45]:
!pip install datetime 

  Obtaining dependency information for datetime from https://files.pythonhosted.org/packages/95/88/3b9d4042b396221a132180b392ab2a174031a6fb579f7927f3909fc183a7/DateTime-5.2-py3-none-any.whl.metadata
     ------------------------------------- 204.1/204.1 kB 12.1 MB/s eta 0:00:00
   ---------------------------------------- 52.2/52.2 kB 2.8 MB/s eta 0:00:00
time: 3.19 s (started: 2023-07-26 19:34:26 +08:00)


In [48]:
from datetime import datetime

time: 0 ns (started: 2023-07-26 19:36:09 +08:00)


In [49]:
# making one big dataframe via pd. concat
big_df= pd.concat(dfs.values(), sort=False).sort_values(['stay_id','delta','label','source'], ascending=True)
#converting delta to time delta, to datetime rounded to 2 minutes, and back to time delta (more efficient than rounding timedeltas)
big_df['delta']= pd.to_timedelta(big_df['delta'])
big_df['delta']= pd.to_datetime(big_df['delta']).dt.round('2min')
big_df['delta']= pd.to_timedelta(big_df['delta'])
#datetime.fromisoformat

TypeError: fromisoformat: argument must be str

time: 20.3 s (started: 2023-07-26 19:36:27 +08:00)


In [50]:
len(big_df)

5959148

time: 0 ns (started: 2023-07-26 19:37:22 +08:00)


In [53]:
big_df.drop_duplicates(subset=['stay_id','label','value','delta',], keep='last', inplace=True) #5959148  -> 5824186 at 2 min. 

time: 2.94 s (started: 2023-07-26 19:38:53 +08:00)


In [54]:
len(big_df)

5824186

time: 0 ns (started: 2023-07-26 19:39:02 +08:00)


In [40]:
#big_df['sum_elix']

time: 429 µs


In [55]:
big_df.groupby('label')['value'].describe() 

,count,unique,top,freq
label,,,,
any_vasoactives,26471.0,2.0,0.0,17954.0
bands,26471,3,absent,24174
bicarbonate,52064.0,96.0,23.0,4217.0
bilirubin,17670.0,606.0,0.3,1261.0
bun,51727.0,334.0,14.0,1524.0
calcium,29730.0,182.0,1.13,1419.0
cancer_elix,26471.0,2.0,0.0,24097.0
chloride,59207.0,137.0,106.0,3564.0
creatinine,51881.0,277.0,0.7,3683.0


time: 1.91 s (started: 2023-07-26 19:39:23 +08:00)


In [42]:
#big_df.groupby('label')['value'].describe() #14478 icustay_id's

time: 467 µs


In [56]:
big_df['label'].unique()

array(['cancer_elix', 'epinephrine', 'gender', 'height', 'norepinephrine',
       'phenylephrine', 'race', 'rrt', 'sum_elix', 'vasopressin',
       'vent_recieved', 'weight', 'yearsold', 'diasbp', 'heartrate',
       'meanartpress', 'o2_flow', 'pao2fio2ratio', 'resprate', 'spo2',
       'sysbp', 'lactate', 'pco2', 'ph', 'po2', 'temperature', 'bands',
       'bicarbonate', 'bilirubin', 'bun', 'chloride', 'creatinine',
       'glucose', 'hemoglobin', 'inr', 'platelet', 'potassium', 'ptt',
       'sodium', 'wbc', 'dobutamine', 'any_vasoactives', 'dopamine',
       'calcium', 'mingcs', 'fio2'], dtype=object)

time: 531 ms (started: 2023-07-26 19:39:31 +08:00)


In [57]:
big_df[big_df['label']=='leukocyte'].head()

,stay_id,t_0,value,label,uom,delta,source,subject_id


time: 391 ms (started: 2023-07-26 19:39:36 +08:00)


## quick qc to protect against old versions

In [58]:
#removing firstpos else neg ssc col if it still exists (qc)
big_df=big_df.loc[:,list(big_df.columns!="first_pos_else_neg_ssc")]

time: 469 ms (started: 2023-07-26 19:40:03 +08:00)


In [59]:
save_df(big_df, 'longdf')

c:\Users\csfhe\Documents\RA\mimiciii-antibiotics-opensource\notebooks/data/cleaned_merged/24_hr_window/
time: 1min 57s (started: 2023-07-26 19:40:04 +08:00)


In [60]:
del big_df
del dfs

time: 297 ms (started: 2023-07-26 19:42:02 +08:00)


#### making a patient missingness visualization

In [48]:
# #big_agg= big_df.groupby(['icustay_id','label'], as_index=False)['value'].agg(['min'])
# big_agg= big_df.groupby(['icustay_id','label'], as_index=False)['value'].size()
# big_agg_count= big_agg.reset_index().pivot(index='icustay_id',columns='label', values=0)#, levels='icustay_id')

time: 1.38 ms


In [49]:
# big_agg_count= big_agg.reset_index().pivot(index='icustay_id',columns='label', values=0)#, levels='icustay_id')

time: 958 µs


In [50]:
# sns.set(rc={'figure.figsize':(25,15)})

time: 1.06 ms


In [51]:
# #big_agg_min

# #%matplotlib inline
# sns.set(rc={'figure.figsize':(25,15)})
# big_agg_count= big_agg_count.fillna(0)
# big_agg_count = big_agg_count[big_agg_count.columns].astype(float) 


# sns.heatmap(big_agg_count,vmin=0, vmax=1, cmap=sns.color_palette("RdBu_r", 5))

time: 2.06 ms


In [52]:
# #deleting df's for memory once saved
# del big_df, big_agg_count, big_agg, dfs

time: 964 µs


In [53]:
# big_agg_count[big_agg_count>0] =1
# big_agg_pt_missing= big_agg_count.T.apply(lambda x:100*(len(list(big_agg_count))-sum(x))/len(list(big_agg_count)))
# big_agg_pt_missing= pd.DataFrame(big_agg_pt_missing).rename(index=str, columns={0:'%_of_values_missing'})

time: 1.1 ms


In [54]:
# big_agg_pt_missing.sort_values('%_of_values_missing',ascending=False).plot()

time: 859 µs
